In [15]:
import torch
import torch.nn as nn
import torch.optim as optimize
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import pandas as pd
from skimage import io
from torch.utils.data import Dataset, DataLoader, ConcatDataset, Subset
from PIL import Image
from sklearn.metrics import classification_report
import random

In [16]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs, L1RegWeight):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        torch.cuda.empty_cache()
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            batches = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                batches += 1
                print("Batch " + str(batches))
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    if (phase != 'val' and L1RegWeight != 0):
                        loss += L1RegWeight * (sum(p.abs().sum() for p in model.parameters()))

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.detach().item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data).detach()

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [17]:
def doValidation(model, dataloader):
    results = {"real": [], "pred": []}
    model.eval()
    torch.set_grad_enabled(False)
    for inputs, labels in dataloader:
        for x in labels.tolist():
            results['real'].append(x)
        inputs = inputs.to(device)
        output = model(inputs)
        for x in torch.argmax(output, dim=1).tolist():
            results["pred"].append(x)
    return results

In [18]:
def initModel(num_classes, batchNormalization):
    if batchNormalization:
        model_ft = models.vgg16_bn(pretrained=True)
    else:
        model_ft = models.vgg16(pretrained=True)
    num_ftrs = model_ft.classifier[6].in_features
    model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
    return model_ft

In [19]:
modelTest = initModel(3, True)

In [20]:
lr = 0.001
momentum = 0.9
regL2_mult = 0.0005

In [21]:
criterion = nn.CrossEntropyLoss()
optim = optimize.SGD(modelTest.parameters(), lr=lr, momentum=momentum, weight_decay=regL2_mult)

In [22]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomCrop(224),
        transforms.ToTensor()
    ]),
    'val': transforms.Compose([
        transforms.Resize([224, 224]),
        transforms.ToTensor()
    ]),
}

In [23]:
class DisastersDatasets(Dataset):
    def __init__(self, csv, root_dir, transform=None):
        self.csv = pd.read_csv(csv, delim_whitespace=True, header=None, usecols=[0,1], names=["img","lbl"])
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.csv)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir, self.csv["img"][idx])

        image = io.imread(img_name)
        image = Image.fromarray(image)
        
        if self.transform:
            image = self.transform(image)
        label = self.csv["lbl"][idx]

        if (image.shape[0] == 4):
            image = image[0:3]

        return image, label

In [24]:
goal_disaster_train_set = DisastersDatasets("ASONAM17_Damage_Image_Dataset\\ecuador.train", "ASONAM17_Damage_Image_Dataset", data_transforms['train'])
test_set = DisastersDatasets("ASONAM17_Damage_Image_Dataset\\ecuador.test", "ASONAM17_Damage_Image_Dataset", data_transforms['val'])
dev_set = DisastersDatasets("ASONAM17_Damage_Image_Dataset\\ecuador.dev", "ASONAM17_Damage_Image_Dataset", data_transforms['val'])
other_disaster_train_set = DisastersDatasets("ASONAM17_Damage_Image_Dataset\\nepal.train", "ASONAM17_Damage_Image_Dataset", data_transforms['train'])
google_train_set = DisastersDatasets("ASONAM17_Damage_Image_Dataset\\gg.train", "ASONAM17_Damage_Image_Dataset", data_transforms['train'])
sizeOfSubset = 500
indices = random.sample(range(0, len(goal_disaster_train_set)), sizeOfSubset)
goal_train_subset = Subset(goal_disaster_train_set, indices)
concatenatedTrainSet = ConcatDataset([google_train_set, goal_train_subset, other_disaster_train_set])

In [25]:
concatenated_train_loader = DataLoader(concatenatedTrainSet, batch_size = 14, shuffle = True)
print("concateSize " + str(len(concatenated_train_loader)))
test_loader = DataLoader(test_set, batch_size = 14, shuffle=True)
dev_loader = DataLoader(dev_set, batch_size = 14, shuffle=True)

concateSize 984


In [26]:
device = torch.device("cuda:0")
modelTest.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [27]:
history = []
for x in range(0, 3):
    torch.cuda.empty_cache()
    modelTest, history_temp = train_model(modelTest, {"train": concatenated_train_loader, "val": test_loader}, criterion, optim, 3, 0)
    history.append(history_temp)
    lr = lr/10
    optim = optimize.SGD(modelTest.parameters(), lr=lr, momentum=momentum, weight_decay=regL2_mult)
torch.cuda.empty_cache()

Epoch 0/2
----------
Batch 1
Batch 2


RuntimeError: CUDA out of memory. Tried to allocate 44.00 MiB (GPU 0; 6.00 GiB total capacity; 4.33 GiB already allocated; 6.50 MiB free; 4.49 GiB reserved in total by PyTorch)

In [14]:
results = doValidation(modelTest, dev_loader)
targetNames = ['none', 'mild', 'severe']
print(results['real'])
print(results['pred'])
print(classification_report(results['real'], results['pred'], target_names=targetNames))

[2, 2, 2, 2, 0, 0, 2, 0, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 1, 0, 2, 0, 0, 0, 2, 1, 0, 2, 1, 2, 2, 2, 2, 0, 1, 0, 2, 2, 0, 0, 2, 0, 0, 0, 2, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 0, 0, 1, 1, 2, 0, 2, 0, 2, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 0, 0, 1, 2, 2, 0, 0, 2, 2, 2, 0, 0, 0, 2, 0, 2, 0, 2, 0, 2, 2, 1, 2, 2, 1, 0, 2, 0, 0, 2, 2, 2, 0, 1, 2, 0, 2, 0, 2, 2, 1, 2, 2, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 1, 2, 0, 0, 1, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 0, 0, 0, 2, 2, 0, 0, 2, 0, 0, 0, 2, 2, 0, 2, 0, 2, 0, 0, 2, 0, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 0, 0, 2, 0, 2, 0, 2, 2, 0, 0, 2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 1, 2, 0, 2, 0, 0, 2, 2, 2, 2, 0, 2, 0, 2, 0, 2, 0, 0, 0, 2, 2, 0, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 2, 2, 0, 1, 2, 0, 0, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 1, 1, 2, 2, 0, 0, 2, 2, 1, 2, 0, 2, 0, 2, 1, 2, 2, 0, 0, 2, 2, 2, 